In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

In [6]:
# === CONFIG ===
DB_PATH = "footbase.db"
DATA_URL = "https://www.football-data.co.uk/mmz4281/2425/E0.csv"
LEAGUE_NAME = "Premier League"
SEASON = "2024/25"

In [7]:
# === 1. EXTRACT ===
print(" Downloading data...")
df = pd.read_csv(DATA_URL)

📥 Downloading data...


In [13]:
# === 2. TRANSFORM ===
print(" Cleaning data...")
df = df.rename(columns={
    'Date': 'date',
    'HomeTeam': 'home_team',
    'AwayTeam': 'away_team',
    'FTHG': 'home_goals',
    'FTAG': 'away_goals',
    'FTR': 'result',
    'HS': 'home_shots',
    'AS': 'away_shots',
    'HST': 'home_shots_on_target',
    'AST': 'away_shots_on_target',
    'B365H': 'odds_home',
    'B365D': 'odds_draw',
    'B365A': 'odds_away'
})

# Keep only relevant columns
keep_cols = [
    'date', 'home_team', 'away_team', 'home_goals', 'away_goals', 'result',
    'home_shots', 'away_shots', 'home_shots_on_target', 'away_shots_on_target',
    'odds_home', 'odds_draw', 'odds_away'
]

df = df[keep_cols]

 Cleaning data...


In [15]:
# Add metadata
df['league'] = LEAGUE_NAME
df['season'] = SEASON

In [16]:
df.head()

,date,home_team,away_team,home_goals,away_goals,result,home_shots,away_shots,home_shots_on_target,away_shots_on_target,odds_home,odds_draw,odds_away,league,season
0,16/08/2024,Man United,Fulham,1,0,H,14,10,5,2,1.60,4.20,5.25,Premier League,2024/25
1,17/08/2024,Ipswich,Liverpool,0,2,A,7,18,2,5,8.50,5.50,1.33,Premier League,2024/25
2,17/08/2024,Arsenal,Wolves,2,0,H,18,9,6,3,1.18,7.50,13.00,Premier League,2024/25
3,17/08/2024,Everton,Brighton,0,3,A,9,10,1,5,2.63,3.30,2.63,Premier League,2024/25
4,17/08/2024,Newcastle,Southampton,1,0,H,3,19,1,4,1.36,5.25,8.00,Premier League,2024/25


In [17]:
# Convert date
try:
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
except Exception:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [19]:
# === 3. LOAD ===
print("💾 Exporting as Tranformed CSV...")
df.to_csv('matches', conn, if_exists='append', index=False)
conn.close()

print(f"✅ Loaded {len(df)} matches into {DB_PATH}")

💾 Loading into SQLite...
✅ Loaded 380 matches into football.db


In [20]:
#Test connection and print to see first 5 rows from the matches table
conn = sqlite3.connect("footbase.db")
df = pd.read_sql("SELECT * FROM matches LIMIT 5", conn)
conn.close()
print(df)

                  date   home_team    away_team  home_goals  away_goals  \
0  2024-08-16 00:00:00  Man United       Fulham           1           0   
1  2024-08-17 00:00:00     Ipswich    Liverpool           0           2   
2  2024-08-17 00:00:00     Arsenal       Wolves           2           0   
3  2024-08-17 00:00:00     Everton     Brighton           0           3   
4  2024-08-17 00:00:00   Newcastle  Southampton           1           0   

  result  home_shots  away_shots  home_shots_on_target  away_shots_on_target  \
0      H          14          10                     5                     2   
1      A           7          18                     2                     5   
2      H          18           9                     6                     3   
3      A           9          10                     1                     5   
4      H           3          19                     1                     4   

   odds_home  odds_draw  odds_away          league   season  
0     